<a href="https://colab.research.google.com/github/chaiyaphum/SuperAI-Engineering-Python-Web-Scraping-Tutorial/blob/main/Python_Web_Scraping_C5_Demo_with_pantip_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install selenium
!pip install selenium

# Update ubuntu to correctly run apt install
!apt-get update

# Install chromedriver
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 1.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [39.3 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://developer.download.nvidia.com/comp

In [6]:
import sys
import os
import time
import requests
from bs4 import BeautifulSoup

import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

driver.get('https://www.pantip.com/')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [8]:
# Click accept popup
popup_accept = driver.find_elements_by_class_name("btn-primary")[0]
popup_accept.click()

MAX_PAGE = 5

counter = 1
thaichana_data = pd.DataFrame([])


while counter <= MAX_PAGE:
    block_popular_loadmore = driver.find_elements_by_class_name("gtm-hitz-more")[0]

    try:
        action = ActionChains(driver)
        action.move_to_element(block_popular_loadmore).perform()
    except:
        print('Out of bounds')
    
    time.sleep(2)
    block_popular_loadmore.click()
    time.sleep(2)
        
    counter = counter + 1


Out of bounds
Out of bounds
Out of bounds
Out of bounds
Out of bounds


In [9]:
pantip_data = pd.DataFrame([])

ul_element = driver.find_elements_by_class_name('pt-list-item__hide-stats-vote')[1]

topic_count = 1

for li_element in ul_element.find_elements_by_tag_name('li'):
    topic_title = li_element.find_elements_by_tag_name('a')[0].text
    topic_url = li_element.find_elements_by_tag_name('a')[0].get_attribute('href')
    
    tag_menu = li_element.find_elements_by_class_name('pt-list-item__tag')[0].find_elements_by_tag_name('a')
    tag_list = [tag.text for tag in tag_menu]

    # Access each topic in list.
    r = requests.get(topic_url)
    topic_soup = BeautifulSoup(r.content, 'html.parser')

    topic_datetime = topic_soup.find('abbr')['data-utime']
    topic_type = topic_soup.find('div', {'class': 'display-post-favourite'})
    topic_type = re.sub('\s+',' ',topic_type.get_text())

    user = topic_soup.find('a', {'class': 'display-post-name'})
    user_id = user['id']
    username = user.get_text()

    if topic_soup.find('div', {'class': 'edit-history'}) is not None:
        topic_soup.find('div', {'class': 'edit-history'}).decompose()
  
    topic_content_raw = topic_soup.find('div', class_='display-post-story')
    topic_content = topic_content_raw.get_text()

    pantip_data = pantip_data.append(pd.DataFrame({
        'topic_title': re.sub('\s+',' ',topic_title),
        'content': re.sub('\s+',' ',topic_content),
        'content_raw': str(topic_content_raw),
        'topic_tags': '|'.join(tag_list),
        'topic_type': topic_type,
        'topic_datetime': topic_datetime,
        'user_id': user_id,
        'username': username,
        'topic_datetime': topic_datetime,
        'topic_url': topic_url
    }, index=[0]), ignore_index=True)

    print("Process topic:", topic_count, topic_title)
    topic_count = topic_count +1

Process topic:  1 One Piece Spoil 993
Process topic:  2 ยอด album gaon รายเดือน เดือนนี้ออกวันไหนค่ะ มีใครพอจะทราบบ้างค่ะ
Process topic:  3 ✿ นังผึ้งหลบไป เพราะนี่คือ นางร้ายตัวแม่ " ร้อยเล่ห์ ... เสน่หา และ ฆาตกรรม " ตัวจริง เสียงจริง ✿
Process topic:  4 วีรยา อายุ 19 โตเป็นสาวแล้ว HBD
Process topic:  5 คนละครึ่ง ค้นหาร้านค้าใกล้ โดยใช้ GPS ครับ
Process topic:  6 เชียร์สด ECZACIBAŞI VİTRA VS GALATASARAY HDI SİGORTA
Process topic:  7 ปิ่น ชรินพร สวย น่ารัก ออร่า ความดังความปังมาก
Process topic:  8 เพลงของ BLACKPINK ที่เกือบจะไม่ได้ปล่อยในครั้งแรก 'WHISTLE' [กระทู้แปลความเห็น K-netizen (ชาวเน็ตเกาหลี)]
Process topic:  9 ร้านค้า คนละครึ่งทำไมหายากจัง
Process topic:  10 "พี่มิวสิค"ครับ ลงรูปงานGrabxBNK48 ได้แล้วหรอครับ
Process topic:  11 ผมว่าอย่างงี้ คุณว่ายังไง BLACKPINK : light up the sky (netflix)
Process topic:  12 Zeng Chulei สละโสด
Process topic:  13 Teaser ฟินมากจากศัตรูสู่หัวใจ
Process topic:  14 ยอด pre-order เสื้องานกีฬาสีอนุบาลหนูน้อย 48
Process topic:  15 จากศัตรูสู่หัวใจ ฉาก

In [11]:
pantip_data.to_csv('example_pantip_data.csv',index=False, encoding="utf-8")